<a href="https://colab.research.google.com/github/Jarcos09/Tareas/blob/main/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎓 **Inteligencia Artificial Aplicada**

## 🤖 **Análisis de grandes volúmenes de datos (Gpo 10)**

### 🏛️ Tecnológico de Monterrey

#### 👨‍🏫 **Profesor titular :** Dr. Iván Olmos Pineda
#### 👩‍🏫 **Profesor asistente :** Verónica Sandra Guzmán de Valle

### 📊 **Proyecto | Base de Datos de Big Data**

#### 📅 **04 de mayo de 2025**

* 🧑‍💻 **A01795941 :** Juan Carlos Pérez Nava




In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('proyectos/librerias'))
if module_path not in sys.path:
    sys.path.append(module_path)
from graficas import *

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, sum, avg, lit, count, when, format_number, round, rand
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import Imputer

import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub

import pandas as pd


from functools import reduce

In [2]:
path = kagglehub.dataset_download("sobhanmoosavi/us-accidents")
print("Path to dataset files:", path)

Path to dataset files: /home/jarcos/.cache/kagglehub/datasets/sobhanmoosavi/us-accidents/versions/13


In [3]:
spark = SparkSession.builder.master("local[*]").appName("CargarCSV").config("spark.driver.memory", "40g").config("spark.executor.memory", "20g").getOrCreate()
df_accident = spark.read.option("header", True).option("inferSchema", True).csv(path)
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/24 00:07:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_accident.show(5)

+---+-------+--------+-------------------+-------------------+-----------------+------------------+-------+-------+------------+--------------------+--------------------+------------+----------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
| ID| Source|Severity|         Start_Time|           End_Time|        Start_Lat|         Start_Lng|End_Lat|End_Lng|Distance(mi)|         Description|              Street|        City|    County|State|   Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|Ameni

**Obtención de las estadísticas descriptivas de las características categóricas**

# Particionamiento

El particionamiento del conjunto de datos se basa en las condiciones climáticas y la severidad del accidente, dividiéndolo en múltiples subconjuntos según combinaciones específicas de estas características.

In [5]:
columnas_clave = [
    "ID", "Weather_Condition","Precipitation(in)","Severity", "City", "State",
    "Temperature(F)", "Humidity(%)", "Visibility(mi)","Wind_Direction","Wind_Speed(mph)","Crossing","Junction","Railway",
    "Roundabout","Stop","Sunrise_Sunset","Traffic_Calming","Traffic_Signal"]

total = df_accident.count()

combinaciones_top = df_accident.groupBy("Weather_Condition", "Severity") \
    .agg(count("*").alias("Frecuencia")) \
    .withColumn("Proporción", col("Frecuencia") / total) \
    .orderBy(col("Proporción").desc())

combinaciones_top = combinaciones_top.withColumn("Frecuencia", col("Frecuencia"))  \
    .withColumn("Proporción", col("Proporción")*100)

df_particionada = df_accident.select(columnas_clave)
df_particionada.write.mode("overwrite").partitionBy("Weather_Condition","Severity").parquet("us_accidents_partitioned")

combinaciones_top.show(10, truncate=False)



+-----------------+--------+----------+------------------+
|Weather_Condition|Severity|Frecuencia|Proporción        |
+-----------------+--------+----------+------------------+
|Fair             |2       |2226576   |28.810332392473782|
|Mostly Cloudy    |2       |792735    |10.25743511523869 |
|Cloudy           |2       |692929    |8.966015449005317 |
|Partly Cloudy    |2       |548760    |7.1005696655734685|
|Clear            |2       |536971    |6.948028270815386 |
|Light Rain       |2       |270162    |3.495706870017238 |
|Overcast         |2       |248938    |3.22108319011686  |
|Clear            |3       |244956    |3.1695589018882835|
|Fair             |3       |240084    |3.1065186376367455|
|Mostly Cloudy    |3       |189229    |2.4484905919651614|
+-----------------+--------+----------+------------------+
only showing top 10 rows



In [6]:
max_reg = 2000

# Filtrar las filas
combinaciones_filtradas = combinaciones_top.filter(col("Frecuencia") >= max_reg)

# Si hay datos, guardarlos en un vector
particiones = combinaciones_filtradas.select("Weather_Condition", "Severity").collect()

# Mostrar el resultado

print(f'✅ Se identificaron \033[32m\033[1m{len(particiones)}\033[0m particiones que contienen más de \033[36m{max_reg}\033[0m registros.')

✅ Se identificaron 77 particiones que contienen más de 2000 registros.


In [7]:
contador_total = 0
semilla = 1854

# Crear un DataFrame vacío con la misma estructura
df_muestras = spark.createDataFrame([], df_particionada.schema)
lista_muestras = []

for particion in particiones:

    contador_total += 1
    weather = particion["Weather_Condition"]
    severity = particion["Severity"]

    print(f"Extrayendo Partición #\033[32m\033[1m{contador_total:03}\033[0m | 🌦 Weather: \033[1;36m{weather}\033[0m | ⚠ Severity: \033[1;36m{severity}\033[0m")

    # Filtrar correctamente la partición
    df_filtrada = df_particionada.filter((col("Weather_Condition") == weather) & (col("Severity") == severity))

    # Limitar a 1200 registros
    df_rand = df_filtrada.orderBy(rand(semilla)).limit(max_reg)

    lista_muestras.append(df_rand)

df_muestras = lista_muestras[0]  # Inicializamos con el primer DataFrame

for df in lista_muestras[1:]:
    df_muestras = df_muestras.union(df)

# **Optimizar con persistencia**
df_muestras = df_muestras.persist().coalesce(8)

# Contar los registros en el nuevo DataFrame
contador_total = df_muestras.count()


print(f"Total de registros obtenidos en la muestra: \033[32m\033[1m{contador_total}\033[0m")

Extrayendo Partición #001 | 🌦 Weather: Fair | ⚠ Severity: 2
Extrayendo Partición #002 | 🌦 Weather: Mostly Cloudy | ⚠ Severity: 2
Extrayendo Partición #003 | 🌦 Weather: Cloudy | ⚠ Severity: 2
Extrayendo Partición #004 | 🌦 Weather: Partly Cloudy | ⚠ Severity: 2
Extrayendo Partición #005 | 🌦 Weather: Clear | ⚠ Severity: 2
Extrayendo Partición #006 | 🌦 Weather: Light Rain | ⚠ Severity: 2
Extrayendo Partición #007 | 🌦 Weather: Overcast | ⚠ Severity: 2
Extrayendo Partición #008 | 🌦 Weather: Clear | ⚠ Severity: 3
Extrayendo Partición #009 | 🌦 Weather: Fair | ⚠ Severity: 3
Extrayendo Partición #010 | 🌦 Weather: Mostly Cloudy | ⚠ Severity: 3
Extrayendo Partición #011 | 🌦 Weather: None | ⚠ Severity: 2
Extrayendo Partición #012 | 🌦 Weather: Scattered Clouds | ⚠ Severity: 2
Extrayendo Partición #013 | 🌦 Weather: Partly Cloudy | ⚠ Severity: 3
Extrayendo Partición #014 | 🌦 Weather: Overcast | ⚠ Severity: 3
Extrayendo Partición #015 | 🌦 Weather: Light Snow | ⚠ Severity: 2
Extrayendo Partición #016 | 

Total de registros obtenidos en la muestra: 148000


# Imputando valores faltantes

In [8]:
def obten_nulos(particion):
  from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

  print(f"📊 Total de filas en la partición: {particion.count()}")
  print(f"🗂️ Número de columnas en la partición: {len(particion.columns)}")

  info_nulos = {}
  cols_nulos = {}

  total_rows = particion.count()

  registros_totales = particion.count()

  # Contar valores nulos por columna

  cols_nulos = particion.select(
    [sum(col(c).isNull().cast("int")).alias(c) for c in particion.columns]
    )

  # Convertir los resultados en un diccionario
  info_nulos = {c: cols_nulos.select(c).collect()[0][0] for c in particion.columns}

  # Filtrar solo las columnas con valores nulos
  cols_nulos = {c: {"count": v, "percent": (v / total_rows) * 100} for c, v in info_nulos.items() if v > 0}

  # Validar si existen nulos
  if not cols_nulos:
        print("✅ No existen valores nulos en la partición.")
        return

  listado = [(key, value['count'], value['percent']) for key, value in cols_nulos.items()]

  # Definir el esquema del DataFrame
  schema = StructType([
    StructField("Columna", StringType(), True),
    StructField("Total de nulos", IntegerType(), True),
    StructField("Porcentaje", DoubleType(), True)
  ])

  df_resumen_nulos = spark.createDataFrame(listado, schema=schema)

  for col_name in [c for c, t in df_resumen_nulos.dtypes if t == "double"]:
      df_resumen_nulos = df_resumen_nulos.withColumn(col_name, round(df_resumen_nulos[col_name], 2))

  df_resumen_nulos.orderBy(col("Total de nulos").desc()).show(truncate=False)


In [9]:
def imputacion_valores(particion):
    print("✅ Se realiza la imputación utilizando los siguientes valores:\n")

    # Obtener las modas (valores más frecuentes)
    moda_Weather = particion.groupBy("Weather_Condition").count().orderBy(col("count").desc()).first()["Weather_Condition"]
    moda_City = particion.groupBy("City").count().orderBy(col("count").desc()).first()["City"]
    moda_Sunset = particion.groupBy("Sunrise_Sunset").count().orderBy(col("count").desc()).first()["Sunrise_Sunset"]
    moda_wub = particion.groupBy("Wind_Direction").count().orderBy(col("count").desc()).first()["Wind_Direction"]

    # Obtener promedios de las variables numéricas
    media_Temperature = particion.select(round(avg(col("Temperature(F)")), 2).alias("avg_temp")).collect()[0][0]
    media_Humidity = particion.select(round(avg(col("Humidity(%)")), 2).alias("avg_humidity")).collect()[0][0]
    media_Visibility = particion.select(round(avg(col("Visibility(mi)")), 2).alias("avg_visibility")).collect()[0][0]
    media_Precipitation = particion.select(round(avg(col("Precipitation(in)")), 2).alias("avg_precipitation")).collect()[0][0]
    media_Wind_Speed = particion.select(round(avg(col("Wind_Speed(mph)")), 2).alias("avg_wind_speed")).collect()[0][0]


    # Imprimir valores calculados correctamente
    print(f"🌡️ Temperatura promedio: {media_Temperature}")
    print(f"💧 Humedad promedio: {media_Humidity}")
    print(f"👀 Visibilidad promedio: {media_Visibility}")
    print(f"🌧️ Precipitación promedio: {media_Precipitation}")
    print(f"🌬️ Velocidad del viento promedio: {media_Wind_Speed}")

    print(f"☁️ Condición meteorológica más frecuente: {moda_Weather}")
    print(f"🏙️ Ciudad más frecuente: {moda_City}")
    print(f"🌅 Hora de atardecer más frecuente: {moda_Sunset}")
    print(f"🌬️ Dirección del viento más frecuente: {moda_wub}")

    # Aplicar imputación con Imputer
    from pyspark.ml.feature import Imputer

    imputer_num = Imputer(
        inputCols=["Temperature(F)", "Humidity(%)", "Visibility(mi)", "Precipitation(in)", "Wind_Speed(mph)"],
        outputCols=["Temperature(F)", "Humidity(%)", "Visibility(mi)", "Precipitation(in)", "Wind_Speed(mph)"]
    ).setStrategy("mean")

    particion = imputer_num.fit(particion).transform(particion)

    # Imputación de valores categóricos con na.fill()
    particion = particion.na.fill({
        "Weather_Condition": moda_Weather,
        "City": moda_City,
        "Sunrise_Sunset": moda_Sunset,
        "Wind_Direction": moda_wub
    })

    print("\n🔍 Se validan nuevamente los valores nulos para corroborar la imputación.\n")

    obten_nulos(particion)

    for col_name in [c for c, t in particion.dtypes if t == "double"]:
      particion = particion.withColumn(col_name, round(particion[col_name], 2))

    return particion

In [10]:
obten_nulos(df_muestras)

📊 Total de filas en la partición: 148000
🗂️ Número de columnas en la partición: 19
+-----------------+--------------+----------+
|Columna          |Total de nulos|Porcentaje|
+-----------------+--------------+----------+
|Precipitation(in)|33645         |22.73     |
|Wind_Speed(mph)  |8824          |5.96      |
|Humidity(%)      |1078          |0.73      |
|Wind_Direction   |726           |0.49      |
|Temperature(F)   |700           |0.47      |
|Sunrise_Sunset   |541           |0.37      |
|Visibility(mi)   |439           |0.3       |
|City             |5             |0.0       |
+-----------------+--------------+----------+



In [11]:
muestra_imp = imputacion_valores(df_muestras)

✅ Se realiza la imputación utilizando los siguientes valores:

🌡️ Temperatura promedio: 57.57
💧 Humedad promedio: 75.21
👀 Visibilidad promedio: 7.02
🌧️ Precipitación promedio: 0.03
🌬️ Velocidad del viento promedio: 10.15
☁️ Condición meteorológica más frecuente: Fair
🏙️ Ciudad más frecuente: Miami
🌅 Hora de atardecer más frecuente: Day
🌬️ Dirección del viento más frecuente: CALM

🔍 Se validan nuevamente los valores nulos para corroborar la imputación.

📊 Total de filas en la partición: 148000
🗂️ Número de columnas en la partición: 19
✅ No existen valores nulos en la partición.


# Preparando datos

In [12]:
muestra_imp.groupBy("Severity").count().show()

+--------+-----+
|Severity|count|
+--------+-----+
|       3|42000|
|       2|74000|
|       4|22000|
|       1|10000|
+--------+-----+



In [13]:
muestra_imp.show()

+---------+-----------------+-----------------+--------+---------------+-----+--------------+-----------+--------------+--------------+---------------+--------+--------+-------+----------+-----+--------------+---------------+--------------+
|       ID|Weather_Condition|Precipitation(in)|Severity|           City|State|Temperature(F)|Humidity(%)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Crossing|Junction|Railway|Roundabout| Stop|Sunrise_Sunset|Traffic_Calming|Traffic_Signal|
+---------+-----------------+-----------------+--------+---------------+-----+--------------+-----------+--------------+--------------+---------------+--------+--------+-------+----------+-----+--------------+---------------+--------------+
|A-6158704|             Fair|              0.0|       2|          Miami|   FL|          74.0|       59.0|          10.0|           ESE|            7.0|   false|   false|  false|     false|false|           Day|          false|         false|
|A-1909088|             Fair|       

In [14]:
categoricas = ["Weather_Condition", "City", "State", "Sunrise_Sunset","Wind_Direction"]
binarias = ["Crossing", "Junction", "Railway", "Roundabout", "Stop", "Traffic_Calming", "Traffic_Signal"]

In [15]:
# ✅ Crear una copia de `imp_sev_1` para trabajar sobre ella
Transf_muestra = muestra_imp.alias("copia_muestra")  # Esto asegura que el original quede intacto

# Convertir variables binarias a 0 y 1 en la copia
for columna in binarias:
    Transf_muestra = Transf_muestra.withColumn(columna + "_num", col(columna).cast("int"))

# Aplicar StringIndexer a las variables categóricas
indexers = [StringIndexer(inputCol=col, outputCol=col + "_Index").fit(Transf_muestra) for col in categoricas]
for indexer in indexers:
    Transf_muestra = indexer.transform(Transf_muestra)

# Aplicar One-Hot Encoding a las categóricas
#codificadores = [OneHotEncoder(inputCol=col + "_Index", outputCol=col + "_OHE").fit(Transf_muestra) for col in categoricas]
#for codificador in codificadores:
#    Transf_muestra = codificador.transform(Transf_muestra)

# 🔥 Eliminar las columnas originales que ya no se usarán en el modelo
Transf_muestra = Transf_muestra.drop(*categoricas).drop(*binarias)

Transf_muestra.show()




+---------+-----------------+--------+--------------+-----------+--------------+---------------+------------+------------+-----------+--------------+--------+-------------------+------------------+-----------------------+----------+-----------+--------------------+--------------------+
|       ID|Precipitation(in)|Severity|Temperature(F)|Humidity(%)|Visibility(mi)|Wind_Speed(mph)|Crossing_num|Junction_num|Railway_num|Roundabout_num|Stop_num|Traffic_Calming_num|Traffic_Signal_num|Weather_Condition_Index|City_Index|State_Index|Sunrise_Sunset_Index|Wind_Direction_Index|
+---------+-----------------+--------+--------------+-----------+--------------+---------------+------------+------------+-----------+--------------+--------+-------------------+------------------+-----------------------+----------+-----------+--------------------+--------------------+
|A-6158704|              0.0|       2|          74.0|       59.0|          10.0|            7.0|           0|           0|          0|     

In [16]:
from pyspark.sql.functions import when, col
from pyspark.sql.functions import mean


calcular_IQR(Transf_muestra,['Temperature(F)','Humidity(%)','Visibility(mi)','Wind_Speed(mph)'])

mean_values = Transf_muestra.select(
    mean("Temperature(F)").alias("mean_temp"),
    mean("Humidity(%)").alias("mean_humidity"),
    mean("Wind_Speed(mph)").alias("mean_wind_speed")
).collect()[0]


Transf_muestra_iqr = Transf_muestra.withColumn("Temperature(F)", when((col("Temperature(F)") < 14.5) | (col("Temperature(F)") > 114.5), mean_values["mean_temp"]).otherwise(col("Temperature(F)"))) \
             .withColumn("Humidity(%)", when((col("Humidity(%)") < -6.0) | (col("Humidity(%)") > 138.0), mean_values["mean_humidity"]).otherwise(col("Humidity(%)"))) \
             .withColumn("Wind_Speed(mph)", when((col("Wind_Speed(mph)") < -2.5) | (col("Wind_Speed(mph)") > 17.5), mean_values["mean_wind_speed"]).otherwise(col("Wind_Speed(mph)")))

In [17]:
calcular_IQR(Transf_muestra,['Temperature(F)','Humidity(%)','Visibility(mi)','Wind_Speed(mph)'])


,IQR,Límite Inf.,Límite Sup.
Columna,,,
Temperature(F),31.4,-5.1,120.5
Humidity(%),30.0,17.0,137.0
Visibility(mi),7.0,-7.5,20.5
Wind_Speed(mph),8.0,-7.0,25.0


In [18]:
calcular_IQR(Transf_muestra_iqr,['Temperature(F)','Humidity(%)','Visibility(mi)','Wind_Speed(mph)'])

,IQR,Límite Inf.,Límite Sup.
Columna,,,
Temperature(F),29.400000,-0.100000,117.500000
Humidity(%),30.000000,17.000000,137.000000
Visibility(mi),7.000000,-7.500000,20.500000
Wind_Speed(mph),5.153283,-2.729925,17.883208


In [19]:
atributos = [ 'Precipitation(in)', 'Temperature(F)', 'Humidity(%)', 'Visibility(mi)',
             'Crossing_num', 'Junction_num', 'Railway_num','Roundabout_num','Stop_num',
              'Traffic_Calming_num', 'Traffic_Signal_num','Weather_Condition_Index',
              'Sunrise_Sunset_Index','Wind_Direction_Index']

In [20]:
assembler = VectorAssembler(inputCols=atributos, outputCol = 'Caracteristicas')
df_vec = assembler.transform(Transf_muestra_iqr)
df_vec.select('Caracteristicas','Severity').show(5,truncate = False)

+----------------------------------------------------------+--------+
|Caracteristicas                                           |Severity|
+----------------------------------------------------------+--------+
|(14,[1,2,3,11,13],[74.0,59.0,10.0,1.0,17.0])              |2       |
|(14,[1,2,3,10,11,12,13],[72.0,87.0,10.0,1.0,1.0,1.0,1.0]) |2       |
|(14,[1,2,3,11,13],[61.0,34.0,10.0,1.0,10.0])              |2       |
|(14,[1,2,3,11,13],[81.0,67.0,10.0,1.0,3.0])               |2       |
|(14,[0,1,2,3,11,12,13],[0.03,46.0,50.0,10.0,1.0,1.0,17.0])|2       |
+----------------------------------------------------------+--------+
only showing top 5 rows



# Creando conjuntos de datos

In [21]:
#spark.conf.set("spark.sql.shuffle.partitions", "200")
train, test = df_vec.randomSplit([0.8,0.2], seed = 10)

train_size = train.count()
test_size = test.count()
total_size = train_size + test_size

train_pct = (train_size / total_size) * 100
test_pct = (test_size / total_size) * 100

print(f"""Existen {train_size} instancias en el conjunto train ({train_pct:.2f}%),
y {test_size} en el conjunto test ({test_pct:.2f}%).""")

Existen 118446 instancias en el conjunto train (80.03%),
y 29554 en el conjunto test (19.97%).


In [22]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol='Caracteristicas', labelCol='Severity', maxIter=10, regParam=0.03, elasticNetParam=0.08)
lr_model = lr.fit(train)

In [23]:
y_pred = lr_model.transform(test)
y_pred.select('Caracteristicas','Severity','prediction').show(truncate = False)

+----------------------------------------------------------+--------+------------------+
|Caracteristicas                                           |Severity|prediction        |
+----------------------------------------------------------+--------+------------------+
|(14,[1,2,3,11,13],[104.0,5.0,10.0,1.0,4.0])               |2       |2.6550031829639167|
|(14,[0,1,2,3,8,11,13],[0.06,71.0,90.0,6.0,1.0,19.0,15.0]) |2       |2.2528655353218925|
|(14,[1,2,3,11,13],[92.0,22.0,10.0,3.0,1.0])               |3       |2.6164156132172742|
|(14,[0,1,2,3,5,11,13],[0.03,64.4,77.0,10.0,1.0,3.0,2.0])  |3       |2.7661285794530857|
|(14,[0,1,2,3,11],[0.03,80.0,87.0,4.0,19.0])               |2       |2.2763153038447306|
|(14,[1,2,3,11,13],[67.0,52.0,10.0,3.0,17.0])              |3       |2.7881919503923642|
|(14,[1,2,3,11,13],[90.0,59.0,10.0,3.0,9.0])               |3       |2.6712980300234914|
|(14,[1,2,3,11,12,13],[71.0,92.0,10.0,3.0,1.0,1.0])        |2       |2.7585024688263315|
|(14,[1,2,3,11],[63.0

In [24]:
print ("The coefficient of the model is : ", lr_model.coefficients)
print ("The Intercept of the model is : ", lr_model.intercept)

The coefficient of the model is :  [0.08756284084703446,-0.0025395143882509168,-0.00019731139566788063,-0.00018495376603987754,-0.2323920555430527,0.08070962194773001,0.0,0.0,-0.15504027215359953,0.0,-0.2780521009097898,-0.022146741276632135,0.10256885115253996,0.007137988708678345]
The Intercept of the model is :  2.9155435604226687


In [25]:
#Root Mean Square Error
eval_lr = RegressionEvaluator(labelCol="Stop_num", predictionCol="prediction", metricName="rmse")
rmse_lr = eval_lr.evaluate(y_pred)
print("RMSE: %.3f" % rmse_lr)

# Mean Square Error
mse = eval_lr.evaluate(y_pred, {eval_lr.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval_lr.evaluate(y_pred, {eval_lr.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval_lr.evaluate(y_pred, {eval_lr.metricName: "r2"})
print("r2: %.3f" %r2)

RMSE: 2.509
MSE: 6.294
MAE: 2.487
r2: -294.737


In [26]:
from pyspark.ml.clustering import KMeans, GaussianMixture
from pyspark.ml.evaluation import ClusteringEvaluator

gmm = GaussianMixture(k=4, seed=1, featuresCol="Caracteristicas", predictionCol="Prediccion")
model = gmm.fit(df_vec)



In [27]:
print("Pesos de los clusters:", model.weights)
print("Medias de los clusters:", [g.mean for g in model.gaussians])
print("Covarianzas de los clusters:", [g.cov for g in model.gaussians])



Pesos de los clusters: [0.8054949460143827, 0.16159496300845294, 0.01153162925286512, 0.021378461724299445]
Medias de los clusters: [DenseVector([0.0243, 57.3572, 75.6449, 6.8598, 0.0002, 0.0977, 0.0001, 0.0, 0.0003, 0.0002, 0.0001, 12.6544, 0.3085, 8.929]), DenseVector([0.0669, 64.0112, 73.2926, 7.5992, 0.5097, 0.0008, 0.0005, 0.0, 0.0013, 0.0011, 0.781, 12.9685, 0.2316, 9.4468]), DenseVector([0.2425, 62.9122, 74.3993, 8.8981, 0.5442, 0.2471, 0.622, 0.0006, 0.0186, 0.0155, 0.3608, 12.9886, 0.2743, 9.515]), DenseVector([0.0311, 59.6231, 73.8734, 7.5877, 0.2817, 0.0061, 0.0036, 0.0003, 0.9901, 0.0084, 0.0615, 12.9917, 0.2939, 8.8088])]
Covarianzas de los clusters: [DenseMatrix(14, 14, [0.0043, 0.0265, 0.2289, -0.0449, 0.0, 0.0004, 0.0, 0.0, ..., -0.0, 0.0, 0.0, -0.0, -0.0, -1.9835, -0.1743, 41.6228], 0), DenseMatrix(14, 14, [0.0379, 0.2236, 1.0306, -0.2125, -0.0213, -0.0, 0.0, -0.0, ..., -0.0003, 0.0002, -0.0005, -0.0007, -0.01, 1.7608, -0.2067, 42.7521], 0), DenseMatrix(14, 14, [0.3768

In [28]:
df_result = model.transform(df_vec)
df_result.select('Caracteristicas','Severity','Prediccion').show(truncate = False)

+----------------------------------------------------------+--------+----------+
|Caracteristicas                                           |Severity|Prediccion|
+----------------------------------------------------------+--------+----------+
|(14,[1,2,3,11,13],[74.0,59.0,10.0,1.0,17.0])              |2       |0         |
|(14,[1,2,3,10,11,12,13],[72.0,87.0,10.0,1.0,1.0,1.0,1.0]) |2       |1         |
|(14,[1,2,3,11,13],[61.0,34.0,10.0,1.0,10.0])              |2       |0         |
|(14,[1,2,3,11,13],[81.0,67.0,10.0,1.0,3.0])               |2       |0         |
|(14,[0,1,2,3,11,12,13],[0.03,46.0,50.0,10.0,1.0,1.0,17.0])|2       |0         |
|(14,[1,2,3,11,13],[51.0,35.0,10.0,1.0,9.0])               |2       |0         |
|(14,[1,2,3,11,12],[56.0,57.0,10.0,1.0,1.0])               |2       |0         |
|(14,[1,2,3,4,10,11,13],[68.0,40.0,10.0,1.0,1.0,1.0,5.0])  |2       |1         |
|(14,[1,2,3,4,11,13],[67.0,70.0,10.0,1.0,1.0,9.0])         |2       |1         |
|(14,[0,1,2,3,11,12,13],[0.0

In [29]:
df_igual = df_result.withColumn("es_igual", when(col("Severity") == col("Prediccion"), 1).otherwise(0))
conteo_iguales = df_igual.select(count(when(col("es_igual") == 1, True)).alias("total_iguales"))

# Mostrar resultados
df_igual.show()
conteo_iguales.show()


+---------+-----------------+--------+--------------+-----------+--------------+---------------+------------+------------+-----------+--------------+--------+-------------------+------------------+-----------------------+----------+-----------+--------------------+--------------------+--------------------+--------------------+----------+--------+
|       ID|Precipitation(in)|Severity|Temperature(F)|Humidity(%)|Visibility(mi)|Wind_Speed(mph)|Crossing_num|Junction_num|Railway_num|Roundabout_num|Stop_num|Traffic_Calming_num|Traffic_Signal_num|Weather_Condition_Index|City_Index|State_Index|Sunrise_Sunset_Index|Wind_Direction_Index|     Caracteristicas|         probability|Prediccion|es_igual|
+---------+-----------------+--------+--------------+-----------+--------------+---------------+------------+------------+-----------+--------------+--------+-------------------+------------------+-----------------------+----------+-----------+--------------------+--------------------+----------------